Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploy Azure ML Hardware Accelerated Models (HAM) to Databox Edge Machine - Using Azure CLI
This notebook will walk through creating and deploying a sample client that can test inferencing of an Azure ML HW Accelerated Model. The sample client image contains a script which uses the AzureML SDK to inference and IoT Hub/Device SDK to send messages to your IoT Hub.

This notebook will walk you through the following:

* [Install dependencies](#installation)
* [Configure your Azure CLI environment](#configuration)
* [Choose image to deploy](#choose-your-model)
* [Create Sample Client image](#sample-client)
* [Deploy modules using CLI](#deploy)
* [Receive IoT Hub Messages using Event Monitor](#iothub-messages)
* [Stop sample client module](#stop-module)
 
## Pre-requisites
* An IoT Edge Device (specifically, the Data Box Edge machine with Compute VM enabled)
    * Follow [steps 1 - 5a](https://docs.microsoft.com/en-us/azure/databox-online/data-box-edge-deploy-prep) to set up your Data Box Edge machine and enable the Linux Compute VM and an associated IoT Hub. This IoT Hub will deploy the AccelContainerImage to the Compute VM. **Note:** In order to use the Azure ML SDK to deploy your model to your IoT Hub, you must create your Data Box Edge resource in the same subscription as your Azure ML Workspace.
* An Azure ML ContainerImage or AccelContainerImage successfully created in your Workspace.
    * For an AccelContainerImage, you can follow [this notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/accelerated-models/accelerated-models-quickstart.ipynb) through Step 5.

<a id='installation'></a>
## Install Azure CLI, IoT extension, and Docker
The following code will:
- install Azure CLI and [Azure IoT CLI extension](https://github.com/Azure/azure-iot-cli-extension)
- install Docker
- sign into your Azure account using browser login

In [ ]:
%%bash
pip install azure-cli # Install Azure CLI
az extension add --name azure-cli-iot-ext # Add IoT CLI extension
pip install -U jupyter_console # Fix Jupyter dependencies overridden by Azure CLI installation
pip install docker # Install Docker
az login # Log into Azure account

<a id='configuration'></a>
## Configure Workspace and IoT information

Next, we will set your default subscription and resource group for accessing your IoT Hub, assuming your IoT Hub is in the same subscription and resource group as your AzureML Workspace. If not, you should update the iot subscription id and resource group.

In [ ]:
from azureml.core import Workspace
import os

ws = Workspace.from_config()

# # If config is not local
# ws = Workspace(subscription_id="", resource_group="", workspace_name="")
# ws.write_config()

ws_subscription_id = ws.subscription_id
ws_resource_group = ws.resource_group

iot_subscription_id = ws.subscription_id # EDIT SUBSCRIPTION ID
iot_resource_group = ws.resource_group # EDIT RESOURCE GROUP

!az account set --subscription $iot_subscription_id
!az configure --defaults group=$iot_resource_group

print("Workspace information:")
print(ws_subscription_id, ws_resource_group, ws.name, ws.location, sep = '\n')
print("------")
print("Using following for IoT Hub and Azure CLI:")
print(iot_subscription_id, iot_resource_group, sep = '\n')

<a id="choose-your-model"></a>
## Configure Azure ML module

List all Images in your Azure ML Workspace.

In [ ]:
from azureml.core.image import Image
for i in Image.list(workspace = ws):
    print('{}'.format(i.image_location))

#### Choose a converted image you'd like to deploy by copying the ACR path below. 
For example, "workspacename1234.azurecr.io/resnet50.1.brainwaveonnx-image:1".

In [ ]:
host_url = "COPY HERE"
acr_name = host_url.split(".")[0]

acr_password = !az acr credential show -n $acr_name --query passwords[0].value  --subscription $ws_subscription_id -g $ws_resource_group
acr_password = acr_password[0].strip("\"")

print("You are choosing to use the Docker image {} in ACR {}.".format(host_url, acr_name))

<a id='sample-client'></a>
## Configure Sample Client module

#### Build sample client image and push to your ACR

The Docker image built from the /sample-client directory will work with the ResNet50 image created in Quickstart. To create a sample client that works with the Image created from transfer learning, update the line to below. The /sample-client uses ImageNet pictures for inferencing; the /sample-client-tl handles the response differently and uses images of cats and dogs. 

```
client_app = "sample-client-tl"
```

In [ ]:
# The default sample client works with the Docker image created from Quickstart
client_app = "sample-client"
version = 1
client_url = "{}.azurecr.io/{}:{}".format(acr_name, client_app, version)

!az acr login --name $acr_name --subscription $ws_subscription_id --resource-group $ws_resource_group
!docker build -t $client_url -f ./Dockerfile ./$client_app
!docker push $client_url

#### Configure Sample Client module
The default input and output tensors are: 
```
input_tensor = "Placeholder:0"
output_tensor = "classifier/resnet_v1_50/predictions/Softmax:0"
```

If you just want to update the input and output tensor names for models other than resnet50 from Quickstart, modify the cell below. 

For example, the transfer learning notebook would use the same input tensor name and change the output tensor name.
```
output_tensor = "classifier_output/Softmax:0"
```

In [ ]:
input_tensor = "Placeholder:0"
output_tensor = "classifier/resnet_v1_50/predictions/Softmax:0"

<a id='deploy'></a>
## Deploy to Databox Edge Machine Using CLI
Here we will utilize the Azure CLI and a pre-defined deployment manifest template which will specify how to deploy your IoT Hub modules. 

When the Azure ML module first deploys, it will take a few minutes to flash the FPGA with the correct firmware for your model type. While this is happening, the sample client module will output that it is waiting for the flashing to finish.

Resources: 
- [az iot edge set-modules](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-deploy-modules-cli)
- [az acr credential show](https://docs.microsoft.com/en-us/cli/azure/acr/credential?view=azure-cli-latest#az-acr-credential-show)
- [az iot hub device-identity show-connection-string](https://docs.microsoft.com/en-us/cli/azure/ext/azure-cli-iot-ext/iot/hub/device-identity?view=azure-cli-latest#ext-azure-cli-iot-ext-az-iot-hub-device-identity-show-connection-string)

### Provide IoT Hub and IoT Edge Device information
When setting up your Compute VM for Databox Edge, you created an IoT Hub and Databox Edge automatically created an IoT Edge device registered to your Compute VM. In order to receive messages in your IoT Hub, copy in your IoT Hub Name and IoT Edge Device name to retrieve your connection string.

In [ ]:
# List all iothubs associated with this resource group
!az iot hub list --output table

#### Copy your IoT Hub name below.

In [ ]:
# Update IoT Hub and Edge Device information
hub_name = "dummy-dbe"

!az iot hub device-identity list --hub-name $hub_name --edge-enabled --output table

In [ ]:
# Copy your DeviceId into the variable below
device_id = "computevm"

### Deploy
#### Create Deployment manifest
The below code will replace all the values we've set previously in a newly generated file called deployment.json.

In [ ]:
device_conn_string = !az iot hub device-identity show-connection-string --hub-name $hub_name --device-id $device_id -o tsv
device_conn_string = device_conn_string[0]

host_status = "running"
client_status = "running"

deployment_manifest_path="./deployment.json"

# Uses global variables
def generate_deployment_manifest():
    file = open("./deployment_template.json")
    contents = file.read()
    contents = contents.replace('$ACR_NAME', acr_name)
    contents = contents.replace('$ACR_PASSWORD', acr_password)
    contents = contents.replace('$INPUT_TENSOR', input_tensor)
    contents = contents.replace('$OUTPUT_TENSOR', output_tensor)
    contents = contents.replace('$HOST_URL', host_url)
    contents = contents.replace('$CLIENT_URL', client_url)
    contents = contents.replace('$DEVICE_CONNECTION_STRING', device_conn_string)
    contents = contents.replace('$HOST_STATUS', host_status)
    contents = contents.replace('$CLIENT_STATUS', client_status)

    with open(deployment_manifest_path, 'wt', encoding='utf-8') as output_file:
        output_file.write(contents)

In [ ]:
generate_deployment_manifest()

#### AzureML Create Container Options
By default, the below is included in the generated deployment.json. If you're using a CPU image you do not need the ``Binds``, ``Privileged``, or ``Devices`` sections and can remove them from the deployment.json file that's generated.
```
{
  "HostConfig": {
    "Binds": [
      "/etc/hosts:/etc/hosts"
    ],
    "Privileged": true,
    "PortBindings": {
      "50051/tcp": [
        {
          "HostPort": "50051"
        }
      ]
    },
    "Devices": [
      {
        "PathOnHost": "/dev/catapult0",
        "PathInContainer": "/dev/catapult0"
      },
      {
        "PathOnHost": "/dev/catapult1",
        "PathInContainer": "/dev/catapult1"
      }
    ]
  }
}
```
#### Sample Client Create Container Options 
By default, the below is included in the generated deployment.json. You can modify it directly in that file if you want to add other parameters or edit the output tensors to return a list (for SSD-VGG). 

Additionally, between each inference call is a wait time of 10 seconds to keep the number of messages to your IoT Hub minimal. You can use the --suppress-messages command or change the --wait time between each inference.
```
{
  "Tty": true,
  "Cmd": [
    "--input-tensors",
    "Placeholder:0",
    "--output-tensors",
    "classifier/resnet_v1_50/predictions/Softmax:0",
    "--wait",
    "10"
  ]
}
```

### Deploy
Now we use the Azure IoT CLI to set the deployment manifest for your Data Box Edge IoT Edge device.

In [ ]:
# This command sets the deployment.json of your device, which includes the settings for both modules
!az iot edge set-modules --device-id $device_id --content $deployment_manifest_path --hub-name $hub_name

<a id='iothub-messages'></a>
## Receive Predictions from IoT Hub
The code below uses the CLI to receive messages from IoT Hub. To use the IoT Service Client SDK, read more here: https://docs.microsoft.com/en-us/azure/iot-hub/iot-hub-devguide-messages-read-builtin

In [ ]:
print("Copy the following command into a bash window:\n")
print("az iot hub monitor-events --hub-name {} --device-id {} --properties app".format(hub_name, device_id))

<a id='stop-module'></a>
## Stop Sample Client Module
We can update the desired status of the IoT Hub modules. Here, we will update the deployment.json to specify that the sample client module should be "stopped". See the [README.md](./README.md) for more information about how to manipulate the sample client module.

In [ ]:
client_status = "stopped"

generate_deployment_manifest()

# This command sets the deployment.json of your device, which includes the settings for both modules
!az iot edge set-modules  --hub-name $hub_name --device-id $device_id --content $deployment_manifest_path